# Spark Assignment

## Bhavana Balakrishna Rao
### 1008221023

In [3]:
!pip install pyspark
!pip install awscli ipython # Optional

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 51.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c274df8f2edf8abeda2a7213731ae8934de6f8696aa534f7d8d674d21f47db5f
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 3.8 MB 9.1 MB/s 
     |████████████████████████████████| 79 kB 9.2 MB/s 
     |████████████████████████████████| 547 kB 67.7 MB/s 
     |████████████████████████████████| 8.7 MB 44.2 MB/s 
     |████████████████████████████████| 138 kB 57.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: rsa
    Found existing installation: rsa 4.8
    Uninstalling rsa-4.8:
      Successfully uninstalled rsa-4.8
  Attempti

In [4]:
from pyspark.sql import Row
from pyspark.sql import SparkSession

import nltk
import nltk.corpus

import collections
import itertools
import os
import string

In [5]:
spark = SparkSession \
    .builder \
    .appName("PySpark demo") \
    .getOrCreate()

Upload dataset_en_train.csv and dataset_en_test.csv.

In [6]:
df = spark.read.json('./dataset*.json')

In [5]:
df.show()

+--------+-------------------+------------------+--------------------+----------+--------------------+-------------------+-----+
|language|   product_category|        product_id|         review_body| review_id|        review_title|        reviewer_id|stars|
+--------+-------------------+------------------+--------------------+----------+--------------------+-------------------+-----+
|      en|          furniture|product_en_0740675|Arrived broken. M...|en_0964290|I'll spend twice ...|reviewer_en_0342986|    1|
|      en|   home_improvement|product_en_0440378|the cabinet dot w...|en_0690095|        Not use able|reviewer_en_0133349|    1|
|      en|               home|product_en_0399702|I received my fir...|en_0311558|The product is junk.|reviewer_en_0152034|    1|
|      en|           wireless|product_en_0444063|This product is a...|en_0044972|Fucking waste of ...|reviewer_en_0656967|    1|
|      en|                 pc|product_en_0139353|went through 3 in...|en_0784379|              bu

In [ ]:
df.count()

66258

In [25]:
from pyspark.sql.functions import col, countDistinct

#df.agg(*(countDistinct(col(c)).alias(c) for c in df.columns))
df.select(countDistinct("review_id")).show()

+-------------------------+
|count(DISTINCT review_id)|
+-------------------------+
|                   179625|
+-------------------------+



Size of the dataframe - (66,258, 8)

In [ ]:
"""rdd = spark.read.json('./dataset_en_train.json').rdd
rdd.flatMap(lambda x: x.review_body.lower().split() + \
        x.review_title.lower().split()) \
   .groupBy(lambda x: x) \
   .map(lambda x: (x[0], len(x[1]))) \
   .sortBy(lambda x: x[1], ascending=False) \
   .take(100)"""

"rdd = spark.read.json('./dataset_en_train.json').rdd\nrdd.flatMap(lambda x: x.review_body.lower().split() +         x.review_title.lower().split())    .groupBy(lambda x: x)    .map(lambda x: (x[0], len(x[1])))    .sortBy(lambda x: x[1], ascending=False)    .take(100)"

In [7]:
def ngram(l, n):
  nltk.download('stopwords', download_dir='/tmp/nltk')
  nltk.data.path.append('/tmp/nltk')
  translation_table = str.maketrans('', '', string.punctuation)
  stopwords = nltk.corpus.stopwords.words('english')
  words = [w.translate(translation_table).lower() for w in l.split() if w.translate(translation_table).lower() not in stopwords]
  d = collections.deque(maxlen=n)
  d.extend(words[:n])
  words = words[n:]
  for window, word in zip(itertools.cycle((d,)), words):
    yield(' '.join(window))
    d.append(word)

In [8]:
def split_grams(n):
  def f(x):
    return [Row(n=n, id=x.review_id, stars=x.stars, tokens=ng) for ng in list(ngram(x.review_title, n))] + \
           [Row(n=n, id=x.review_id, stars=x.stars, tokens=ng) for ng in list(ngram(x.review_body, n))]
  return f

In [9]:
#grams = df.rdd.flatMap(lambda x: split_grams(1)(x) + split_grams(2)(x) + split_grams(3)(x))
#grams.toDF().registerTempTable("tokens")

# using 1-, 2- and 3-word sequences
grams = df.rdd.flatMap(lambda x: split_grams(1)(x) + split_grams(2)(x) + split_grams(3)(x))
grams.toDF().registerTempTable("table")

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [47]:
spark.sql(""" 
SELECT *
FROM tokens
WHERE n=2
""").show(truncate=False)

+---+----------+-----+-------------------+
|n  |id        |stars|tokens             |
+---+----------+-----+-------------------+
|2  |en_0964290|1    |ill spend          |
|2  |en_0964290|1    |spend twice        |
|2  |en_0964290|1    |twice amount       |
|2  |en_0964290|1    |amount time        |
|2  |en_0964290|1    |time boxing        |
|2  |en_0964290|1    |boxing whole       |
|2  |en_0964290|1    |whole useless      |
|2  |en_0964290|1    |useless thing      |
|2  |en_0964290|1    |thing send         |
|2  |en_0964290|1    |send back          |
|2  |en_0964290|1    |back 1star         |
|2  |en_0964290|1    |1star review       |
|2  |en_0964290|1    |arrived broken     |
|2  |en_0964290|1    |broken manufacturer|
|2  |en_0964290|1    |manufacturer defect|
|2  |en_0964290|1    |defect two         |
|2  |en_0964290|1    |two legs           |
|2  |en_0964290|1    |legs base          |
|2  |en_0964290|1    |base completely    |
|2  |en_0964290|1    |completely formed  |
+---+------

### TF-IDF for negative reviews

In [48]:
# for n=1
spark.sql(""" 
WITH t1 as (
  SELECT id, 
    CASE WHEN stars = 1 OR stars = 2 THEN "negative" ELSE "positive" END AS sentiment,
    tokens
  FROM table
  WHERE n=1 AND (stars = 1 OR stars = 2)
),
t2 as (
  SELECT
    id, sentiment, tokens,
    COUNT(tokens) OVER (PARTITION BY id, tokens) as token_freq,
    COUNT(*) OVER (PARTITION BY id) as total_tokens_in_doc
  FROM t1
),
t3 as (
  SELECT id, tokens, token_freq, total_tokens_in_doc,
	CAST(token_freq AS DECIMAL) / CAST(total_tokens_in_doc AS DECIMAL) as tf
	FROM t2
),
t4 AS(
	SELECT DISTINCT tokens, COUNT(DISTINCT id) AS docs_with_token -- no. of documents with the given keyword (denominator in idf)
	FROM t3
	GROUP BY tokens
),
t5 as (
  SELECT t3.*, LOG((SELECT COUNT(DISTINCT id) FROM t3)/ t4.docs_with_token) AS idf
	FROM t3
	LEFT JOIN t4
	ON t3.tokens = t4.tokens
)
SELECT *, tf*idf AS tf_idf
FROM t5
GROUP BY tokens, id, token_freq, total_tokens_in_doc, tf, idf
ORDER BY tf_idf DESC
""").show(truncate=False)

+----------+---------------+----------+-------------------+-------------+------------------+------------------+
|id        |tokens         |token_freq|total_tokens_in_doc|tf           |idf               |tf_idf            |
+----------+---------------+----------+-------------------+-------------+------------------+------------------+
|en_0560040|junkdoesn’t    |1         |1                  |1.00000000000|11.313742551100862|11.313742551100862|
|en_0034742|expectedbut    |1         |1                  |1.00000000000|11.313742551100862|11.313742551100862|
|en_0716472|junkwaste      |1         |1                  |1.00000000000|11.313742551100862|11.313742551100862|
|en_0997052|spears         |1         |1                  |1.00000000000|11.313742551100862|11.313742551100862|
|en_0072464|antidepressants|1         |1                  |1.00000000000|11.313742551100862|11.313742551100862|
|en_0249014|cheapbreaks    |1         |1                  |1.00000000000|11.313742551100862|11.313742551

In [49]:
# for n=2
spark.sql(""" 
WITH t1 as (
  SELECT id, 
    CASE WHEN stars = 1 OR stars = 2 THEN "negative" ELSE "positive" END AS sentiment,
    tokens
  FROM table
  WHERE n=2 AND (stars = 1 OR stars = 2)
),
t2 as (
  SELECT
    id, sentiment, tokens,
    COUNT(tokens) OVER (PARTITION BY id, tokens) as token_freq,
    COUNT(*) OVER (PARTITION BY id) as total_tokens_in_doc
  FROM t1
),
t3 as (
  SELECT id, tokens, token_freq, total_tokens_in_doc,
	CAST(token_freq AS DECIMAL) / CAST(total_tokens_in_doc AS DECIMAL) as tf
	FROM t2
),
t4 AS(
	SELECT DISTINCT tokens, COUNT(DISTINCT id) AS docs_with_token -- no. of documents with the given keyword (denominator in idf)
	FROM t3
	GROUP BY tokens
),
t5 as (
  SELECT t3.*, LOG((SELECT COUNT(DISTINCT id) FROM t3)/ t4.docs_with_token) AS idf
	FROM t3
	LEFT JOIN t4
	ON t3.tokens = t4.tokens
)
SELECT *, tf*idf AS tf_idf
FROM t5
GROUP BY tokens, id, token_freq, total_tokens_in_doc, tf, idf
ORDER BY tf_idf DESC
""").show(truncate=False)

+----------+---------------------------+----------+-------------------+-------------+------------------+------------------+
|id        |tokens                     |token_freq|total_tokens_in_doc|tf           |idf               |tf_idf            |
+----------+---------------------------+----------+-------------------+-------------+------------------+------------------+
|en_0283526|arrived dusty              |1         |1                  |1.00000000000|11.295900655914194|11.295900655914194|
|en_0689412|billed never               |1         |1                  |1.00000000000|11.295900655914194|11.295900655914194|
|en_0907324|application start          |1         |1                  |1.00000000000|11.295900655914194|11.295900655914194|
|en_0806063|advertised nit             |1         |1                  |1.00000000000|11.295900655914194|11.295900655914194|
|en_0779167|0 stari                    |1         |1                  |1.00000000000|11.295900655914194|11.295900655914194|
|en_0902

In [10]:
# for n=3
spark.sql(""" 
WITH t1 as (
  SELECT id, 
    CASE WHEN stars = 1 OR stars = 2 THEN "negative" ELSE "positive" END AS sentiment,
    tokens
  FROM table
  WHERE n=3 AND (stars = 1 OR stars = 2)
),
t2 as (
  SELECT
    id, sentiment, tokens,
    COUNT(tokens) OVER (PARTITION BY id, tokens) as token_freq,
    COUNT(*) OVER (PARTITION BY id) as total_tokens_in_doc
  FROM t1
),
t3 as (
  SELECT id, tokens, token_freq, total_tokens_in_doc,
	CAST(token_freq AS DECIMAL) / CAST(total_tokens_in_doc AS DECIMAL) as tf
	FROM t2
),
t4 AS(
	SELECT DISTINCT tokens, COUNT(DISTINCT id) AS docs_with_token -- no. of documents with the given keyword (denominator in idf)
	FROM t3
	GROUP BY tokens
),
t5 as (
  SELECT t3.*, LOG((SELECT COUNT(DISTINCT id) FROM t3)/ t4.docs_with_token) AS idf
	FROM t3
	LEFT JOIN t4
	ON t3.tokens = t4.tokens
)
SELECT *, tf*idf AS tf_idf
FROM t5
GROUP BY tokens, id, token_freq, total_tokens_in_doc, tf, idf
ORDER BY tf_idf DESC
""").show(truncate=False)

+----------+--------------------+----------+-------------------+-------------+------------------+------------------+
|id        |tokens              |token_freq|total_tokens_in_doc|tf           |idf               |tf_idf            |
+----------+--------------------+----------+-------------------+-------------+------------------+------------------+
|en_0786258|1200 lm like        |1         |1                  |1.00000000000|11.252261753742095|11.252261753742095|
|en_0727514|6 days get          |1         |1                  |1.00000000000|11.252261753742095|11.252261753742095|
|en_0155072|2 years starting    |1         |1                  |1.00000000000|11.252261753742095|11.252261753742095|
|en_0481820|6 cans description  |1         |1                  |1.00000000000|11.252261753742095|11.252261753742095|
|en_0967461|1 3 gave            |1         |1                  |1.00000000000|11.252261753742095|11.252261753742095|
|en_0693305|3 month already     |1         |1                  |

## Discussion of Results

The TF-IDF for 1-, 2-, 3- n-grams were calculated. Each review is considered as a document and each n-gram is considered as a term. The results above shows the different n-grams that best represent negative reviews. Some of these phrases include 'arrived dusty', 'failed', '3 weeks yet'.

The top 10% from the above results can be chosen as blacklisted keywords to identify negative reviews which can ultimately lead to increased customer satisfaction.